In [3]:
import os
import json
import numpy as np
import ollama  # Importar la API de Ollama
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine

# Cargar el modelo de embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Parámetros de filtrado
UMBRAL_BASE = 0.60  
UMBRAL_MAXIMO = 0.45  
MIN_DOCUMENTOS_RELEVANTES = 1

def cargar_embeddings_desde_archivo(ruta_archivo):
    """Carga los embeddings desde un archivo JSON."""
    with open(ruta_archivo, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data["archivo"], data["secciones"]

def calcular_similitud(embedding1, embedding2):
    """Calcula la similitud mediante la distancia del coseno."""
    return cosine(embedding1, embedding2)

def buscar_documentos_similares(texto, carpeta_embeddings):
    """
    Busca documentos en una carpeta que sean más similares al texto de entrada.
    Retorna una lista con el nombre del archivo, la menor distancia y el párrafo más similar.
    """
    embedding_texto = embedding_model.encode(texto)
    resultados = {}

    for archivo in os.listdir(carpeta_embeddings):
        if archivo.endswith("_embeddings.json"):
            ruta_archivo = os.path.join(carpeta_embeddings, archivo)
            nombre_archivo, secciones = cargar_embeddings_desde_archivo(ruta_archivo)

            menor_distancia = float("inf")
            parrafo_mas_similar = ""

            for seccion in secciones:
                embedding_parrafo = np.array(seccion["embedding"])
                distancia = calcular_similitud(embedding_texto, embedding_parrafo)

                if distancia < menor_distancia:
                    menor_distancia = distancia
                    parrafo_mas_similar = seccion["texto"]

            resultados[nombre_archivo] = (menor_distancia, parrafo_mas_similar)

    resultados_ordenados = sorted(resultados.items(), key=lambda x: x[1][0])

    umbral_actual = UMBRAL_BASE
    documentos_relevantes = [(archivo, distancia, parrafo) for archivo, (distancia, parrafo) in resultados_ordenados if distancia <= umbral_actual]

    if len(documentos_relevantes) < MIN_DOCUMENTOS_RELEVANTES:
        umbral_actual = UMBRAL_MAXIMO
        documentos_relevantes = [(archivo, distancia, parrafo) for archivo, (distancia, parrafo) in resultados_ordenados if distancia <= umbral_actual]

    if len(documentos_relevantes) < MIN_DOCUMENTOS_RELEVANTES:
        return []

    return documentos_relevantes

def generar_respuesta_con_ollama(texto_pregunta, carpeta_embeddings, modelo_ollama="mistral"):
    """
    Genera una respuesta en lenguaje natural a partir de los párrafos más similares,
    utilizando Ollama como modelo generativo e incluyendo referencias a los documentos originales.
    """
    documentos_relevantes = buscar_documentos_similares(texto_pregunta, carpeta_embeddings)

    if not documentos_relevantes:
        return "No se encontró una respuesta clara en los documentos."

    # Construir el prompt para Ollama
    contexto = "A continuación, se presentan extractos de documentos relevantes:\n\n"
    referencias = []

    for archivo, distancia, parrafo in documentos_relevantes[:3]:  # Tomar hasta 3 párrafos relevantes
        contexto += f"- {parrafo}\n\n"
        referencias.append(f"{archivo} (distancia: {distancia:.4f})")

    contexto += f"\nPregunta: {texto_pregunta}\n"
    contexto += "Por favor, genera una respuesta concisa basada en la información proporcionada."

    # Enviar el prompt a Ollama
    respuesta_ollama = ollama.chat(model=modelo_ollama, messages=[{"role": "user", "content": contexto}])

    # Extraer solo el contenido de la respuesta
    respuesta_generada = respuesta_ollama["message"]["content"]

    # Incluir referencias en la salida
    respuesta_final = f"{respuesta_generada}\n\n📌 Referencias utilizadas:\n" + "\n".join(referencias)

    return respuesta_final

# Ejemplo de uso
carpeta_embeddings = "data"
texto_pregunta = "All the information about Burkina?"

respuesta = generar_respuesta_con_ollama(texto_pregunta, carpeta_embeddings, modelo_ollama="mistral")

# Mostrar la respuesta generada
print("\n🔹 Respuesta generada:")
print(respuesta)



🔹 Respuesta generada:
 La información proporcionada indica que el HRP (Humanitarian Response Plan) de Burkina Faso está bajofinanciado, y hay dudas sobre si un proyecto piloto de acción anticipatorio puede desviar fondos necesarios para necesidades más urgentes identificadas por el Cluster de Seguridad Alimentaria. La pregunta sugerida es: ¿Cuál podría ser el impacto potencial de $15 millones en la región o a nivel global, y cómo se compara con los impactos del enfoque sin riesgos?

📌 Referencias utilizadas:
1e8c2332a461a3a142840fa477fa907c66c35dac (distancia: 0.5305)
